# Mini Project 3 - Decision Tree

## Introduction

In this project, we implement a Decision Tree for classification. A decision tree is a predictive model that maps observations (features) to conclusions (target labels) by recursively splitting the data into more homogeneous subsets. For our datasets, which include the Iris (3 classes) and Spambase (binary classification) datasets, the splits are binary and are determined by maximizing the **information gain** based on the reduction in **entropy**.

**Key Concepts:**

- **Entropy:** A measure of the impurity or uncertainty in the target variable.  
  $$ H(Y) = -\sum_{i} p_i \log_2(p_i) $$

- **Information Gain:** The reduction in entropy achieved by splitting a node.  
  $$ IG = H(\text{parent}) - \left(\frac{N_{left}}{N_{parent}} H(\text{left}) + \frac{N_{right}}{N_{parent}} H(\text{right})\right) $$

- **Early Stopping with n_min:** Instead of growing the tree until pure leaves, we stop splitting if a node contains fewer than a threshold number of samples n_min, defined as a percentage of the training set.


For the Iris dataset, we will try n\_min ∈ {5, 10, 15, 20} and use 10-fold cross-validation. For Spambase, we will use n\_min ∈ {5, 10, 15, 20, 25}.


In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt

## Node Representation

We define a `Node` class to represent a decision point in our tree. Each node has the following attributes:

- **feature_name**: (or index) The feature used for splitting at that node.
- **threshold**: The numeric threshold used for the split (e.g., "petal_length < 2.5").
- **left** and **right**: The left and right child nodes, respectively.  
  (The left child corresponds to samples where the feature value is less than or equal to the threshold.)
- **is_leaf**: A Boolean flag indicating whether the node is a leaf.
- **value**: If the node is a leaf, this stores the predicted class label.


In [2]:
# Class for each node in the decision tree
class Node():
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, is_leaf = False, value = None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.is_leaf = is_leaf
        self.value = value

## Decision Tree Class 

In [3]:
# Class for the Decision Tree model
class DecisionTree():
    def __init__(self, n_min = 5, training_size=None):
        self.n_min = n_min # Minimum number of samples required to split a node
        self.training_size = training_size
        self.root = None

    def compute_entropy(self, y):
        counts = np.bincount(y.astype(int)) # Count occurrences of each class
        probabilities = counts / len(y)     # Convert to probabilities
        entropy = 0

        for p in probabilities:
            if p > 0:
                entropy += -p * math.log2(p)

        return entropy

    def compute_information_gain(self, feature_column, y, threshold):
        original_entropy = self.compute_entropy(y)

        # Split data into left and right based on threshold
        left_mask = (feature_column <= threshold)
        right_mask = (feature_column > threshold)

        y_left = y[left_mask]
        y_right = y[right_mask]

        n = len(y)
        n_left = len(y_left)
        n_right = len(y_right)

        if n_left == 0 or n_right == 0:
            return 0  # If one child is empty, no gain.

        # Compute "weighted" entropy after split
        child_entropy = ((n_left / n) * self.compute_entropy(y_left)) + ((n_right / n) * self.compute_entropy(y_right))
        
        return original_entropy - child_entropy

    def choose_best_split(self, X, y):
        best_gain = -1 # Initalizing gain to a large -ve value
        best_feature_index = None
        best_threshold = None

        n_samples, n_features = X.shape  

        for feature_index in range(n_features):
            values = np.unique(X[:, feature_index])  # Get unique feature values
            values.sort() # Sort values to create threshold candidates

            if len(values) < 2:
                continue # Skip if there is no possible split
            
            for i in range(len(values) - 1):
                threshold = (values[i] + values[i + 1]) / 2  # The midpoints of consecutive unique values are taken as thresholds
                gain = self.compute_information_gain(X[:, feature_index], y, threshold)

                if gain > best_gain:
                    best_gain = gain
                    best_feature_index = feature_index
                    best_threshold = threshold

        return best_feature_index, best_threshold

    def create_tree(self, X, y, min_samples_leaf):
        # Stop if all samples belong to one class or too few samples to split
        if len(np.unique(y)) == 1 or len(y) < min_samples_leaf:
            counts = np.bincount(y.astype(int))   # Count class occurrences
            majority_label = np.argmax(counts)    # Assign majority class label
            return Node(value = majority_label, is_leaf = True)   # Create leaf node

        best_feature, best_threshold = self.choose_best_split(X, y)   # Find best split

        if best_feature is None:
            counts = np.bincount(y.astype(int))
            majority_label = np.argmax(counts)
            return Node(value = majority_label, is_leaf = True)   # Create leaf node

        node = Node(feature_index = best_feature, threshold = best_threshold, is_leaf = False)   # Create decision node

        # Split data into left and right based on best feature and threshold
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        if len(y_left) == 0 or len(y_right) == 0:
            counts = np.bincount(y.astype(int))
            majority_label = np.argmax(counts)
            return Node(value = majority_label, is_leaf = True)   # Create leaf node

        # Recursively create left and right child nodes
        node.left = self.create_tree(X_left, y_left, min_samples_leaf )
        node.right = self.create_tree(X_right, y_right, min_samples_leaf )

        return node

    def fit(self, X, y):
        min_samples_leaf = math.ceil((self.n_min / 100) * len(X))
        self.root = self.create_tree(X, y, min_samples_leaf)

    def predict_sample(self, x, node):
        if node.is_leaf:
            return node.value   # Return class label if leaf node
        else:
            if x[node.feature_index] <= node.threshold:
                return self.predict_sample(x, node.left)   # Go left if condition is met
            else:
                return self.predict_sample(x, node.right)  # Otherwise, go right

    def predict(self, X):
        n = X.shape[0]
        predictions = []
        
        for i in range(n):
            sample = X[i]
            prediction = self.predict_sample(sample, self.root)
            predictions.append(prediction)
        
        return np.array(predictions)

## Model Evaluation on the Iris Dataset

We now perform 10-fold cross-validation to test our decision tree for various values of n_min. We record the mean accuracy and standard deviation for each n_min value.

In [4]:
# Load Iris dataset
iris_df = pd.read_csv("iris.csv", names=["sepal_length", "sepal_width", "petal_length", "petal_width", "class"])
with pd.option_context('future.no_silent_downcasting', True):
    iris_df= iris_df.replace({'Iris-setosa':0, 'Iris-versicolor': 1, 'Iris-virginica': 2}).infer_objects()

X = iris_df.iloc[:, :-1].values  
y = iris_df.iloc[:, -1].values  

In [5]:
# 10-fold cross-validation
kf = KFold(n_splits = 10, shuffle=True, random_state=42)
n_min_vals = {5, 10, 15, 20}
results = []

for n_min in n_min_vals:
    accuracy_vals = []
    for train_index, test_index in kf.split(X):
        # Splitting the dataset into train set and test set
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train Decision Tree with current n_min
        tree = DecisionTree(n_min = n_min)
        tree.fit(X_train, y_train)

        # Make predictions
        y_pred = tree.predict(X_test)

        # Compute accuracy
        acc = accuracy_score(y_test, y_pred)
        accuracy_vals.append(acc)

    # Compute mean and standard deviation of accuracy
    avg_accuracy = np.mean(accuracy_vals)
    std_accuracy = np.std(accuracy_vals)

    results.append({"n_min": n_min, "Mean Accuracy": avg_accuracy, "Std Dev": std_accuracy})

In [6]:
results_df = pd.DataFrame(results)
results_df

,n_min,Mean Accuracy,Std Dev
0,10,0.946667,0.049889
1,20,0.946667,0.049889
2,5,0.946667,0.049889
3,15,0.946667,0.049889


## Model Evaluation on the Spambase Dataset

Repeat a similar evaluation using the Spambase dataset. Use n_min values from {5, 10, 15, 20, 25} and perform 10-fold cross-validation.


In [7]:
# Load Spambase dataset
spambase_df = pd.read_csv("spambase.csv")

X = spambase_df.iloc[:, :-1].values  
y = spambase_df.iloc[:, -1].values

In [8]:
# 10-fold cross-validation
kf = KFold(n_splits = 10, shuffle=True, random_state=42)
n_min_vals = {5, 10, 15, 20, 25}
results = []

for n_min in n_min_vals:
    accuracy_vals = []
    for train_index, test_index in kf.split(X):
        # Splitting the dataset into train set and test set
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Train Decision Tree with current n_min
        tree = DecisionTree(n_min = n_min)
        tree.fit(X_train, y_train)

        # Make predictions
        y_pred = tree.predict(X_test)

        # Compute accuracy
        acc = accuracy_score(y_test, y_pred)
        accuracy_vals.append(acc)

    # Compute mean and standard deviation of accuracy
    avg_accuracy = np.mean(accuracy_vals)
    std_accuracy = np.std(accuracy_vals)

    results.append({"n_min": n_min, "Mean Accuracy": avg_accuracy, "Std Dev": std_accuracy})

In [9]:
results_df = pd.DataFrame(results)
results_df

,n_min,Mean Accuracy,Std Dev
0,20,0.858913,0.020450
1,5,0.901522,0.019687
2,25,0.827609,0.016529
3,10,0.888696,0.014544
4,15,0.868478,0.023216


## Conclusion & Discussion

- The decision tree was built recursively by choosing the best split based on information gain.
- Early stopping was applied using the n_min threshold to prevent overfitting.
- We evaluated our model using 10-fold cross-validation on both the Iris and Spambase datasets.
- The results (mean accuracy and standard deviation) are summarized in the tables above.